In [4]:
import pyodbc
import os, json
import pandas as pd
import datetime
from datetime import datetime, timedelta

arquivo_json = os.path.join(os.environ['USERPROFILE'], "config_robos.json")

with open(arquivo_json, encoding='utf-8') as f:
    dados = json.load(f)

usuarioID = dados["senha_rede"]["chave"]
senhaAcesso = dados["senha_rede"]["senha"]



# Defina suas credenciais e propriedades de conexão
def new_func(usuarioID, senhaAcesso, strSQL):
    host = "vdados.petrobras.com.br"
    port = 9401
    username = usuarioID
    password = senhaAcesso
    database = "S10267"  # O dataSource

    connection = pyodbc.connect('dsn=TDVCPBIP')



    strSQL = strSQL
    df = pd.read_sql(strSQL, connection)

    return df



In [5]:
# Calcula a data formatada como hoje - 35 dias
dataFormatada = (datetime.now() - timedelta(days=35)).strftime('%Y-%m-%d')
strSQL = f"""
SELECT 
    ESQUEMA.OBJNR AS OBJETO_ORDEM, 
    STD.TXT04 AS STATUS, 
    STD.TXT30 AS TEXTO_STATUS, 
    ST.USNAM AS USUARIO_MODIFICACAO_STATUS, 
    CAST(ST.UDATE AS DATE) AS DATA_MODIFICACAO_STATUS, 
    CAST(ST.UTIME AS TIME) AS HORA_MODIFICACAO_STATUS, 
    ST.INACT AS INDICADOR_STATUS_INATIVO, 
    ST.CHIND AS INDICADOR_TIPO_MODIFICACAO, 
    ST._CHANGE_TIME AS DATA_HORA_ATUALIZACAO_ODS
FROM 
    DADOS_BRUTOS.K_PM_MANUTENCAO.JSTO AS ESQUEMA
INNER JOIN 
    DADOS_BRUTOS.K_PM_MANUTENCAO.JCDS AS ST ON ESQUEMA.OBJNR = ST.OBJNR
INNER JOIN 
    DADOS_BRUTOS.K_PM_MANUTENCAO.TJ30T AS STD 
    ON ST.STAT = STD.ESTAT AND ESQUEMA.STSMA = STD.STSMA
INNER JOIN (
    SELECT DISTINCT O2.OBJNR
    FROM DADOS_BRUTOS.K_PM_MANUTENCAO.AUFK AS O2
    INNER JOIN DADOS_BRUTOS.K_PM_MANUTENCAO.JCDS AS ST2 ON O2.OBJNR = ST2.OBJNR
    WHERE 
        O2.WERKS = '1400' AND 
        O2.AEDAT > '{dataFormatada}' AND 
        ST2.UDATE > '{dataFormatada}'
) AS FILTRADOS ON ESQUEMA.OBJNR = FILTRADOS.OBJNR
WHERE 
    STD.TXT04 IN ('FELE', 'FINT', 'FENG', 'FCAL', 'FMEC', 'DLEX', 'FAPL', 'CPXC', 'PTTC') AND 
    ST.UDATE > '{dataFormatada}'
"""

new_func(usuarioID, senhaAcesso, strSQL)

C:\Users\D25E\AppData\Local\Temp\ipykernel_28292\2787186065.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strSQL, connection)


,OBJETO_ORDEM,STATUS,TEXTO_STATUS,USUARIO_MODIFICACAO_STATUS,DATA_MODIFICACAO_STATUS,HORA_MODIFICACAO_STATUS,INDICADOR_STATUS_INATIVO,INDICADOR_TIPO_MODIFICACAO,DATA_HORA_ATUALIZACAO_ODS
0,OR002028117629,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-04,17:04:30,,I,2025-03-17 13:02:24.161
1,OR002029804488,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:55:53,,U,2025-03-17 13:02:24.171
2,OR002029834745,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:54:28,,U,2025-03-17 13:02:24.179
3,OR002029804486,CPXC,Complexidade de planej Baixa,A1VQ,2025-02-14,16:15:06,,U,2025-03-17 13:02:24.179
4,OR002029835966,CPXC,Complexidade de planej Baixa,RXTS,2025-01-06,12:10:28,,U,2025-03-17 13:02:24.175
...,...,...,...,...,...,...,...,...,...
12548,OR002030452846,DLEX,Delineamento Executado,EBGK,2025-04-24,13:38:33,,I,2025-04-24 13:38:35.248
12549,OR002029936700,DLEX,Delineamento Executado,DQH1,2025-01-07,10:42:10,,I,2025-03-17 13:02:24.140
12550,OR002030459063,DLEX,Delineamento Executado,ASB5,2025-04-28,09:56:06,,I,2025-04-28 09:56:20.027
12551,OR002029719186,DLEX,Delineamento Executado,G0A8,2025-04-09,12:20:09,,I,2025-04-09 12:20:12.494


In [6]:
strSQL = f"""SELECT TOP 10 MANDT, AUFNR, ARTPR, PRIOK, EQUNR, BAUTL, ILOAN, ILOAI, ANLZU, IWERK, INGPR, APGRP, PM_OBJTY, GEWRK, KUNUM, ANING, GAUZT, GAUEH, ANLBD, ANLVD, ANLBZ, ANLVZ, INSPK, DATAN, WARPL, ABNUM, WAPOS, LAUFN, OBKNR, REVNR, ADDAT, ADUHR, IPHAS, ILART, QMNUM, HISDA, AKKNZ, PLKNZ, SERIALNR, SERMAT, DEVICEID, SCREENTY, ADPSP, RSUPG, CHANGEDDATETIME, MAINTORDPERSONRESPONSIBLE, MAINTORDOVRLPROCPHASE, MAINTORDOVRLPROCSUBPHASE, DUMMY_AFIH_INCL_EEW_PS, "_ISDFPS_OBJNR", "_ISDFPS_MEQUI", "_ISDFPS_MHIO_ADDATE", "_ISDFPS_MHIO_ADTIME", "_ISDFPS_USERMODE", UII, DATAMODELVERSION, SERVICE_DOC_TYPE, SERVICE_DOC_ID, SERVICE_DOC_ITEM_ID, IS_BILLABLE, FLDLOGSDELIVISHELDONSHORE, ORCOD, PRMAN, PRVAL, TWTDE, MSTCK, LACD_DATE, OLD_LACD_DATE
FROM DADOS_BRUTOS.K_PM_MANUTENCAO.AFIH;"""

new_func(usuarioID, senhaAcesso, strSQL)



C:\Users\D25E\AppData\Local\Temp\ipykernel_28292\2787186065.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(strSQL, connection)


,MANDT,AUFNR,ARTPR,PRIOK,EQUNR,BAUTL,ILOAN,ILOAI,ANLZU,IWERK,...,SERVICE_DOC_ITEM_ID,IS_BILLABLE,FLDLOGSDELIVISHELDONSHORE,ORCOD,PRMAN,PRVAL,TWTDE,MSTCK,LACD_DATE,OLD_LACD_DATE
0,400,003005489615,ST,,000000000030106501,,000100891842,X,,0301,...,000000,,,,,00,,,00000000,00000000
1,400,003005489616,ST,,000000000030106501,,000100891843,X,,0301,...,000000,,,,,00,,,00000000,00000000
2,400,003005489617,ST,,000000000030106501,,000100891844,X,,0301,...,000000,,,,,00,,,00000000,00000000
3,400,003005489634,ST,,000000000030107208,,000100891882,X,,0301,...,000000,,,,,00,,,00000000,00000000
4,400,003005489636,ST,,000000000030107208,,000100891884,X,,0301,...,000000,,,,,00,,,00000000,00000000
5,400,003005489639,ST,,000000000030107208,,000100891887,X,,0301,...,000000,,,,,00,,,00000000,00000000
6,400,003005489640,ST,,000000000030107208,,000100891888,X,,0301,...,000000,,,,,00,,,00000000,00000000
7,400,003005489641,ST,,000000000030107208,,000100891889,X,,0301,...,000000,,,,,00,,,00000000,00000000
8,400,003005489642,ST,,000000000030107208,,000100891891,X,,0301,...,000000,,,,,00,,,00000000,00000000
9,400,003005489643,ST,,000000000030107208,,000100891892,X,,0301,...,000000,,,,,00,,,00000000,00000000
